In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def download_pdfs(pdf_links, folder_name):
    # Download each PDF
    for pdf_url in pdf_links:
        pdf_name = pdf_url.split("/")[-1]
        file_path = os.path.join(folder_name, pdf_name)

        if os.path.exists(file_path):
            print(f"✅ Already downloaded: {pdf_name}")
            continue

        try:
            pdf_res = requests.get(pdf_url)
            pdf_res.raise_for_status()
            with open(file_path, "wb") as f:
                f.write(pdf_res.content)
            print(f"📥 Downloaded: {pdf_name}")
        except Exception as e:
            print(f"❌ Failed to download {pdf_name}: {e}")

    # Optional: Delay to be polite
    time.sleep(1)

In [6]:
import os
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

# Folder to save PDFs
os.makedirs("/content/drive/MyDrive/Projects/RagTaxHelper/irs_eng_pdfs", exist_ok=True)
os.makedirs("/content/drive/MyDrive/Projects/RagTaxHelper/irs_multilang_pdfs", exist_ok=True)

base_url = "https://www.irs.gov"
start_url = "https://www.irs.gov/downloads/irs-pdf?page={}"

headers = {
    "User-Agent": "Mozilla/5.0"
}

for page_num in range(58):  # Pages 0 to 57
    print(f"\n🔍 Scraping page {page_num}...")
    url = start_url.format(page_num)

    try:
        res = requests.get(url, headers=headers)
        res.raise_for_status()
    except Exception as e:
        print(f"❌ Failed to fetch page {page_num}: {e}")
        continue

    soup = BeautifulSoup(res.text, "html.parser")

    #Find all elements from the table
    table = soup.find("table", attrs={"class": "table table-striped tablesaw tablesaw-stack cols-4"})

    tbody = table.find("tbody", recursive=False)
    form_rows = tbody.find_all("tr", recursive=False)

    single_language_regex = r"\((sp|SP|ru|RU|vie|VIE|zh-s|ZH-S|zh-t|ZH-T|de|DE|ko|KO|ht|HT)\)+"
    filtered_forms = []
    for form in form_rows:
            form_desc_element = form.find("td", attrs={"headers": "view-name-table-column"})
            form_desc = form_desc_element.contents[0]
            if not bool(re.search(single_language_regex, form_desc)):
                filtered_forms.append(form)

    multi_language_regex = r"\(en-(sp|SP|ru|RU|vie|VIE|zh-s|ZH-S|zh-t|ZH-T|de|DE|ko|KO|ht|HT)\)+"

    english_forms = []
    multilang_forms = []

    for form in filtered_forms:
            form_desc_element = form.find("td", attrs={"headers": "view-name-table-column"})
            form_desc = form_desc_element.contents[0]
            if not bool(re.search(multi_language_regex, form_desc)):
                english_forms.append(form.find("td", attrs={"headers": "view-uri-table-column"}))
            else:
                multilang_forms.append(form.find("td", attrs={"headers": "view-uri-table-column"}))

    eng_pdf_links = [
            urljoin(base_url, form.find("a")["href"])
            for form in english_forms
        ]

    multilang_pdf_links = [
            urljoin(base_url, form.find("a")["href"])
            for form in multilang_forms
        ]

    download_pdfs(eng_pdf_links, "/content/drive/MyDrive/Projects/RagTaxHelper/irs_eng_pdfs")
    download_pdfs(multilang_pdf_links, "/content/drive/MyDrive/Projects/RagTaxHelper/irs_multilang_pdfs")

print("\n🎉 Done downloading all PDFs.")



🔍 Scraping page 0...
📥 Downloaded: p6079.pdf
📥 Downloaded: f433f.pdf
📥 Downloaded: f433h.pdf
📥 Downloaded: f1096.pdf
📥 Downloaded: f1097btc.pdf
📥 Downloaded: i1098.pdf
📥 Downloaded: p2104.pdf
📥 Downloaded: p4299.pdf
📥 Downloaded: p5838.pdf
📥 Downloaded: p6026.pdf
📥 Downloaded: f15111.pdf
📥 Downloaded: p5911.pdf
📥 Downloaded: f15112.pdf
📥 Downloaded: p4444.pdf
📥 Downloaded: p5726-2025-03-00.pdf
📥 Downloaded: p6090.pdf
📥 Downloaded: p5379.pdf
📥 Downloaded: p6087.pdf
📥 Downloaded: i1099sa.pdf
📥 Downloaded: i1099ls.pdf
📥 Downloaded: i8991.pdf
📥 Downloaded: p6012.pdf
📥 Downloaded: p6088.pdf
📥 Downloaded: i3921.pdf
📥 Downloaded: i1099mec.pdf
📥 Downloaded: p3.pdf
📥 Downloaded: f1099r.pdf
📥 Downloaded: p5341.pdf
📥 Downloaded: f13614c.pdf
📥 Downloaded: f1098.pdf
📥 Downloaded: f1099h.pdf
📥 Downloaded: i1099sb.pdf
📥 Downloaded: i1097btc.pdf
📥 Downloaded: p5883.pdf
📥 Downloaded: p6081.pdf
📥 Downloaded: f1098q.pdf
📥 Downloaded: p4190.pdf
📥 Downloaded: f1099sa.pdf
📥 Downloaded: f1099a.pdf
📥 Downloa

In [7]:
import shutil

# Copy folder to a new location in your Google Drive
shutil.copytree('irs_eng_pdfs', "/content/drive/MyDrive/Projects/RagTaxHelper/irs_eng_pdfs")
shutil.copytree('irs_multilang_pdfs', '"/content/drive/MyDrive/Projects/RagTaxHelper/irs_multilang_pdfs")


SyntaxError: unterminated string literal (detected at line 5) (<ipython-input-7-1a9ed30c4f8f>, line 5)

In [ ]:
import os
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Folder to save PDFs
os.makedirs("irs_all_pdfs", exist_ok=True)

base_url = "https://www.irs.gov"
start_url = "https://www.irs.gov/downloads/irs-pdf?page={}"

headers = {
    "User-Agent": "Mozilla/5.0"
}

for page_num in range(58):  # Pages 0 to 57
    print(f"\n🔍 Scraping page {page_num}...")
    url = start_url.format(page_num)

    try:
        res = requests.get(url, headers=headers)
        res.raise_for_status()
    except Exception as e:
        print(f"❌ Failed to fetch page {page_num}: {e}")
        continue

    soup = BeautifulSoup(res.text, "html.parser")

    # Find all <a> tags with href ending in .pdf
    links = soup.find_all("a", href=True)

print(links[0])